In [ ]:
#dependencies
import requests
import pandas as pd
import numpy as np
import json
import time
import matplotlib.pyplot as plt
from scipy.stats import linregress
from config import api_key
import seaborn as sns
season = 2022

In [ ]:
#get team ids
team_url = f'http://api.sportradar.us/nba/trial/v8/en/seasons/{season}/REG/standings.json?api_key={api_key}'

team_data = requests.get(team_url).json()
def extract_team_data(data):
    teams = []
    if 'teams' in data:
        for team in data['teams']:
            team_id = team['id']
            team_name = team['name']
            teams.append({'id': team_id, 'name': team_name})
    if 'divisions' in data:
        for division in data['divisions']:
            teams += extract_team_data(division)
    if 'conferences' in data:
        for conference in data['conferences']:
            teams += extract_team_data(conference)
    return teams

# Extract team id and team name from the team_data dictionary
teams = extract_team_data(team_data)
teams_info=pd.DataFrame(teams)
teams_id= teams_info["id"]
time.sleep(1)

In [ ]:
# Initialize
#team lists
team_name=[]
team_possession=[]
team_assists=[]
team_field_goals=[]
team_id=[]
team_data={}
team_offensive_rebounds=[]
team_turnovers=[]
team_free_throws_att=[]
opp_possession=[]
possession=[]
team_o_rtg=[]
team_d_rtg=[]
team_net_rtg=[]
field_goals_made=[]
three_points_att=[]
three_points_made=[]
free_throws_made=[]
ow_defensive_rebounds=[]
ow_points=[]

#opponent vs team lists
opp_points=[]
opp_defensive_rebounds=[]
opp_offensive_rebounds=[]
opp_field_goals_made=[]
opp_three_points_made=[]
opp_free_throws_made=[]
opp_free_throws_att=[]
opp_field_goals_att=[]
opp_three_points_att=[]
opp_turnovers=[]
i=0
# Loop through each team ID and retrieve their statistics from the API
for i in range(len(teams_id)):
    team_url = f'http://api.sportradar.us/nba/trial/v8/en/seasons/{season}/REG/teams/{teams_id[i]}/statistics.json?api_key={api_key}'
    team_data = requests.get(team_url).json()
    # Extract various statistics for the each team
    field_goals_att=team_data['own_record']['total']['field_goals_att']
    makes=team_data['own_record']['total']['field_goals_made']
    offensive_rebounds=team_data['own_record']['total']['offensive_rebounds']
    defensive_rebounds=team_data['own_record']['total']['defensive_rebounds']
    ft_att=team_data['own_record']['total']['free_throws_att']
    turnovers=team_data['own_record']['total']['turnovers']
    free_throws_att=team_data['own_record']['total']['free_throws_att']
    points=team_data['own_record']['total']['points']
    
    offensive_rebounds_opp=team_data['opponents']['total']['offensive_rebounds']
    defensive_rebounds_opp=team_data['opponents']['total']['defensive_rebounds']
    field_goals_att_opp=team_data['opponents']['total']['field_goals_att']
    makes_opp=team_data['opponents']['total']['field_goals_made']
    ft_att_opp=team_data['opponents']['total']['free_throws_att']
    turnovers_opp=team_data['opponents']['total']['turnovers']
    points_opp=team_data['opponents']['total']['points']

    team_totals_dict = team_data['own_record']['total']
    opp_vs_team_totals_dict = team_data['opponents']['total']

    team_stat_keys = ['games_played', 'minutes', 'field_goals_made', 'field_goals_att',
                      'three_points_made', 'three_points_att', 'three_points_pct',
                      'free_throws_made', 'free_throws_att', 'offensive_rebounds',
                      'defensive_rebounds', 'rebounds', 'assists', 'turnovers',
                      'steals', 'blocks', 'points']
    team_stats = {k: v for k, v in team_totals_dict.items() if k in team_stat_keys}
    opp_vs_team_stats = {k: v for k, v in opp_vs_team_totals_dict.items() if k in team_stat_keys}

    #team stats calculations
    #new possession formula
    team_poss = field_goals_att + (0.4 * ft_att) - (1.07 * ((offensive_rebounds) / (offensive_rebounds + defensive_rebounds_opp)) * (field_goals_att - makes)) + (turnovers)
    opp_possession = field_goals_att_opp + (0.4 * ft_att_opp) - (1.07 * ((offensive_rebounds_opp) / (offensive_rebounds_opp + defensive_rebounds)) * (field_goals_att_opp - makes_opp)) + (turnovers_opp)
    possession = (team_poss + opp_possession) / 2
    
    o_rtg = 100 * (points) / (possession)
    d_rtg = 100 * (points_opp) / (possession)
    net_rtg = (o_rtg) - (d_rtg)

    #Append the extracted statistics to teams' lists
    team_name.append(team_data['name'])
    team_id.append(team_data['id']) 
    team_field_goals.append(field_goals_att)
    field_goals_made.append(team_stats["field_goals_made"])
    team_assists.append(team_stats['assists'])
    team_offensive_rebounds.append(offensive_rebounds)
    team_turnovers.append(turnovers)
    team_free_throws_att.append(free_throws_att)
    team_possession.append(possession)
    team_o_rtg.append(o_rtg)
    team_d_rtg.append(d_rtg)
    team_net_rtg.append(net_rtg)
    three_points_att.append(team_stats['three_points_att'])
    three_points_made.append(team_stats["three_points_made"])
    free_throws_made.append(team_stats["free_throws_made"])
    ow_defensive_rebounds.append(team_stats["defensive_rebounds"])
    ow_points.append(team_stats['points'])

    #opponents vs
    opp_offensive_rebounds.append(opp_vs_team_stats['offensive_rebounds'])
    opp_defensive_rebounds.append(opp_vs_team_stats["defensive_rebounds"])
    opp_points.append(opp_vs_team_stats['points'])
    opp_field_goals_att.append(opp_vs_team_stats['field_goals_att'])
    opp_field_goals_made.append(opp_vs_team_stats['field_goals_made'])
    opp_three_points_att.append(opp_vs_team_stats['three_points_att'])
    opp_three_points_made.append(opp_vs_team_stats['three_points_made'])
    opp_free_throws_att.append(opp_vs_team_stats['free_throws_att'])
    opp_free_throws_made.append(opp_vs_team_stats['free_throws_made'])
    opp_turnovers.append(opp_vs_team_stats['turnovers'])
    


    # Pause execution for 0.5 second to avoid overloading the API
    time.sleep(1)

# Create Teams DataFrame
teams_df=pd.DataFrame({
    'Team Name' : team_name,
    'Team Possession': team_possession,
    'team_o_rtg': team_o_rtg,
    'team_d_rtg': team_d_rtg,
    'team_net_rtg': team_net_rtg,
    #team   
    'Team Field Goals Attempts':team_field_goals,
    'Team Field Goals Made':field_goals_made,
    'Team Assists': team_assists,
    'Team Offensive Rebounds': team_offensive_rebounds,
    'Team Defensive Rebounds': ow_defensive_rebounds,
    'Team Turnovers': team_turnovers,
    'Team Free throws Attemps': team_free_throws_att,
    'Team Free throws Made': free_throws_made,
    'Team Three Points Attempts': three_points_att,
    'Team Three Points Made':three_points_made,
    'Team Points': ow_points,

    #opponents
    'Opp Ponits': opp_points,
    'Opp Defensive Rebounds': opp_defensive_rebounds,
    'Opp Offensive Rebounds': opp_offensive_rebounds,
    'Opp Turnovers': opp_turnovers,
    'Opp Field Goals Attempts': opp_field_goals_att,
    'Opp Field Goals Made': opp_field_goals_made,
    'Opp Free throws Attemps': opp_free_throws_att,
    'Opp Free throws Made': opp_free_throws_made,
    'Opp Three Points Attempts': opp_three_points_att,
    'Opp Three Points Made': opp_three_points_made
   

})
teams_df

In [ ]:
#get standings and wins
standing_url = f'http://api.sportradar.us/nba/trial/v8/en/seasons/{season}/REG/standings.json?api_key={api_key}'
standing_data = requests.get(standing_url).json()
aa= standing_data["conferences"]
Teams_name=[]
standing_rank=[]
conferences2=[]
win=[]
n=i=j=0

for n in range(len(aa)):
    zz= standing_data["conferences"][n]['divisions']
    
    
    for i in range(len(zz)):
        bb=standing_data["conferences"][n]['divisions'][i]['teams']
        

        for j in range(len(bb)):
            Teams_name.append(standing_data["conferences"][n]['divisions'][i]['teams'][j]['name'])
            standing_rank.append(standing_data["conferences"][n]['divisions'][i]['teams'][j]['calc_rank']['conf_rank'])
            win.append(standing_data["conferences"][n]['divisions'][i]['teams'][j]['wins'])
            conferences2.append(standing_data["conferences"][n]['alias'])
    #b=pd.DataFrame[b]
    standing_df=pd.DataFrame({"Team Name": Teams_name,
                'Wins': win,
                'Conference Ranking': standing_rank,
                'Conferences':conferences2

   })
standing_df

#b5
last_team_df=pd.merge(standing_df,teams_df,on="Team Name")
last_team_df=last_team_df.sort_values('Conference Ranking')
last_team_df

In [ ]:
#more calculations

#pace = possesions / games (82 for a typical regular season)
last_team_df['Pace'] = last_team_df['Team Possession'] / 82

#AST% = 'Team Assists' / 'Team Field Goals Made'
last_team_df['AST%'] = last_team_df['Team Assists'] / last_team_df['Team Field Goals Made']

#3 point rate (3PR) = 'Team Three Points Attempts' / 'Team Field Goals Attempts'
last_team_df['3PR'] = last_team_df['Team Three Points Attempts'] / last_team_df['Team Field Goals Attempts']

#true shooting percentage = 'Team Points' / 2 * ('Team Field Goals Attempts' + 0.44 * 'Team Free throws Attempts')
last_team_df['TS%'] = last_team_df['Team Points'] / 2 * (last_team_df['Team Field Goals Attempts'] + 0.44 * last_team_df['Team Free throws Attempts'])

#point differential = 'Team Points' - 'Opp Points'
last_team_df['Point Differential'] = last_team_df['Team Points'] - last_team_df['Opp Points']

#relative net rating = team net rating - average net rating for all teams
last_team_df['rNet Rtg'] = last_team_df['team_net_rtg'] - last_team_df['team_net_rtg'].mean()

#relative true shooting percentage = Team TS% - leage average TS%
last_team_df['rTS%'] = last_team_df['TS%'] - last_team_df['TS%'].mean()




In [ ]:
#Dean Oliver's four factors
#effective field goal percentage (eFG%)
#formula = ('Team Field Goals made' + 0.5 * ' Team Three Points Made') / 'Team Field Goals Attempts'
#team
last_team_df['eFG%'] = (last_team_df['Team Field Goals Made'] + (0.5 * last_team_df['Team Three Points Made'])) / last_team_df['Team Field Goals Attempts']

#opponents
last_team_df['Opp eFG%'] = (last_team_df['Opp Field Goals Made'] + (0.5 * last_team_df['Opp Three Points Made'])) / last_team_df['Opp Field Goals Attempts']


In [ ]:
#turnover percentage (TOV%)
#formula = 'Team Turnovers' / ('Team Field Goals Made' + (0.44 * 'Team Free throws Made') + 'Team Turnovers')
#team
last_team_df['Team TOV%'] = last_team_df['Team Turnovers'] / (last_team_df['Team Field Goals Made'] + (0.44 * last_team_df['Team Free throws Made']) + last_team_df['Team Turnovers'])

#opponents
last_team_df['Opp TOV%'] = last_team_df['Opp Turnovers'] / (last_team_df['Opp Field Goals Made'] + (0.44 * last_team_df['Opp Free throws Made']) + last_team_df['Opp Turnovers'])


In [ ]:
#offensive rebound rate (ORR%)
#formula = 'Team Offensive Rebounds' / ('Team Offensive Rebounds' + 'Opp Defensive Rebounds')
#team
last_team_df['Team ORR%'] = last_team_df['Team Offensive Rebounds'] / (last_team_df['Team Offensive Rebounds'] + last_team_df['Opp Defensive Rebounds'])

#opponents
last_team_df['Opp ORR%'] = last_team_df['Opp Offensive Rebounds'] / (last_team_df['Opp Offensive Rebounds'] + last_team_df['Team Defensive Rebounds'])


In [ ]:
#free throw rate (FTR)
#formula = 'Team Free throws Made' / 'Team Field Goals Attempts'
#team
last_team_df['Team FTR'] = last_team_df['Team Free throws Made'] / last_team_df['Team Field Goals Attempts']

#opponents
last_team_df['Opp FTR'] = last_team_df['Opp Free throws Made'] / last_team_df['Opp Field Goals Attempts']


four_factors_df = last_team_df.loc[:, ('Team Name', 'Conferences', 'Wins', 'Pace', '3PR', 'team_net_rtg', 'team_o_rtg', 'team_d_rtg', 'Team eFG%', 'Team TOV%', 'Team ORR%', 'Team FTR', 'Opp eFG%', 'Opp TOV%', 'Opp ORR%', 'Opp FTR')]


In [ ]:
#Combining the four factors for teams and opponents into one number for graphing
#team - opponent shooting
four_factors_df['Net eFG%'] = (four_factors_df['Team eFG%'] - four_factors_df['Opp eFG%']) 

#opponent - team turnover rate
four_factors_df['Net TOV%'] = (four_factors_df['Opp TOV%'] - four_factors_df['Team TOV%'])

#team - opponent o reb rate
four_factors_df['Net ORR%'] = (four_factors_df['Team ORR%'] - four_factors_df['Opp ORR%'])

#team - opponent free throw rate
four_factors_df['Net FTR'] = (four_factors_df['Team FTR'] - four_factors_df['Opp FTR'])
#combined
four_factors_df['Four Factors Score'] = (four_factors_df['Net eFG%'] * 0.4) + (four_factors_df['Net TOV%'] * 0.25) + (four_factors_df['Net ORR%'] * 0.2) + (four_factors_df['Net FTR'] * 0.15)


In [ ]:
#conference stats
conference_groups = four_factors_df.groupby('Conferences')

conference_dfs = {}
for conference, conference_df in conference_groups:
    conference_dfs[conference] = conference_df


eastern_conf_df = conference_dfs['EASTERN']
western_conf_df = conference_dfs['WESTERN']

eastern_conf_df


# Calculate the mean of each column in the eastern conference dataframe
eastern_conf_means = eastern_conf_df.mean()

# Create a new row with label "conference average" and assign the calculated means to each column
eastern_conf_df.loc['Conference average'] = eastern_conf_means

# Calculate the mean of each column in the western conference dataframe
western_conf_means = western_conf_df.mean()

# Create a new row with label "conference average" and assign the calculated means to each column
western_conf_df.loc['Conference average'] = western_conf_means

eastern_conf_df

In [ ]:
#visualizations
# four factors box and whiskers chart across conferences
conf_df = four_factors_df[['Conferences', 'Net eFG%', 'Net TOV%', 'Net ORR%', 'Net FTR', '3PR', 'Pace', 'team_net_rtg', 'team_d_rtg', 'team_o_rtg', 'Four Factors Score']]
conf_four_factors_df = pd.melt(conf_df, id_vars=['Conferences'], value_vars=['Net eFG%', 'Net TOV%', 'Net ORR%', 'Net FTR'])

fig, ax = plt.subplots()
sns.boxplot(x='variable', y='value', hue='Conferences', data=conf_four_factors_df, ax=ax, palette=['blue', 'red'])

ax.set_title('Comparison of Conference Metrics: Four Factors')

plt.show()


In [ ]:
conf_ratings_df = pd.melt(conf_df, id_vars=['Conferences'], value_vars=['team_o_rtg', 'team_d_rtg'])

fig, ax = plt.subplots()
sns.boxplot(x='variable', y='value', hue='Conferences', data=conf_ratings_df, ax=ax, palette=['blue', 'red'])


ax.set_title('Comparison of Conference Metrics: Offensive and Defensive Rating')


# Show the plot
plt.show()

In [ ]:
conf_net_df = pd.melt(conf_df, id_vars=['Conferences'], value_vars=['team_net_rtg'])
fig, ax = plt.subplots()
sns.boxplot(x='variable', y='value', hue='Conferences', data=conf_net_df, ax=ax, palette=['blue', 'red'])


ax.set_title('Comparison of Conference Metrics: Net Rating')


# Show the plot
plt.show()

In [ ]:
conf_3pr_df = pd.melt(conf_df, id_vars=['Conferences'], value_vars=['3PR'])
# Create the box plot using matplotlib
fig, ax = plt.subplots()
sns.boxplot(x='variable', y='value', hue='Conferences', data=conf_3pr_df, ax=ax, palette=['blue', 'red'])


ax.set_title('Comparison of Conference Metrics: Three-point Rate')

# Show the plot
plt.show()

In [ ]:
conf_pace_df = pd.melt(conf_df, id_vars=['Conferences'], value_vars=['Pace'])
# Create the box plot using matplotlib
fig, ax = plt.subplots()
sns.boxplot(x='variable', y='value', hue='Conferences', data=conf_pace_df, ax=ax, palette=['blue', 'red'])


ax.set_title('Comparison of Conference Metrics: Pace')


# Show the plot
plt.show()

In [ ]:
conf_four_factors_score_df = pd.melt(conf_df, id_vars=['Conferences'], value_vars=['Four Factors Score'])

fig, ax = plt.subplots()
sns.boxplot(x='variable', y='value', hue='Conferences', data=conf_four_factors_score_df, ax=ax, palette=['blue', 'red'])

ax.set_title('Comparison of Conference Metrics: Weighted Average of Four Factors')

plt.show()

In [ ]:
#wins vs net rating
plt.scatter(last_team_df["team_net_rtg"], last_team_df["Wins"], edgecolor="Blue", alpha=1)

plt.xlabel("Net Rating")
plt.ylabel("Wins")


plt.grid(True)
x_values=last_team_df["team_net_rtg"]
y_values=last_team_df["Wins"]
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
r_values= x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.plot(x_values,r_values,"r-")
plt.annotate(line_eq,(30,114),fontsize=15,color="red")
print("The r-values is: "+ str(rvalue))
plt.show()

In [ ]:
#wins vs offense
plt.scatter(last_team_df["team_o_rtg"], last_team_df["Wins"], edgecolor="Blue", alpha=1)

plt.xlabel("Offensive Rating")
plt.ylabel("Wins")


plt.grid(True)
x_values=last_team_df["team_o_rtg"]
y_values=last_team_df["Wins"]
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
r_values= x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.plot(x_values,r_values,"r-")
plt.annotate(line_eq,(30,114),fontsize=15,color="red")
print("The r-values is: "+ str(rvalue))
plt.show()

In [ ]:
#wins vs defense
plt.scatter(last_team_df["team_d_rtg"], last_team_df["Wins"], edgecolor="Blue", alpha=1)

plt.xlabel("Defensive Rating")
plt.ylabel("Wins")


plt.grid(True)
x_values=last_team_df["team_d_rtg"]
y_values=last_team_df["Wins"]
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
r_values= x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.plot(x_values,r_values,"r-")
plt.annotate(line_eq,(30,114),fontsize=15,color="red")
print("The r-values is: "+ str(rvalue))
plt.show()

In [ ]:
#four factors vs wins
plt.scatter(four_factors_df['Four Factors Score'], four_factors_df["Wins"], edgecolor="Blue", alpha=1)

plt.xlabel('Four Factors Score')
plt.ylabel('Wins')


plt.grid(True)
x_values=four_factors_df['Four Factors Score']
y_values=four_factors_df['Wins']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
r_values= x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.plot(x_values,r_values,"r-")
plt.annotate(line_eq,(20,0),fontsize=15,color="red")
print(f'r value: {rvalue}')

plt.show()

In [ ]:
#point differential vs wins
plt.scatter(last_team_df['Point Differential'], four_factors_df['Wins'], edgecolor='Blue', alpha=1)

plt.xlabel('Point Differential')
plt.ylabel('Wins')


plt.grid(True)
x_values=last_team_df['Point Differential']
y_values=four_factors_df['Wins']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
r_values= x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.plot(x_values,r_values,"r-")
plt.annotate(line_eq,(20,0),fontsize=15,color="red")
print(f'r value: {rvalue}')

plt.show()

In [ ]:
#rNet Rtg vs wins
plt.scatter(last_team_df['rNet Rtg'], four_factors_df['Wins'], edgecolor='Blue', alpha=1)

plt.xlabel('Net Rating relative to league average')
plt.ylabel('Wins')


plt.grid(True)
x_values=last_team_df['rNet Rtg']
y_values=four_factors_df['Wins']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
r_values= x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.plot(x_values,r_values,"r-")
plt.annotate(line_eq,(20,0),fontsize=15,color="red")
print(f'r value: {rvalue}')

plt.show()
